
# Analyse de radiosondage sur émagramme

**Auteur : Frédéric FERRY (ENM/C3M) - Novembre 2021**

En météorologie, on représente l’état de l’atmosphère (pression, température, humidité) et les transformations subies par les particules d’air sur des diagrammes thermodynamiques. Plusieurs diagrammes sont utilisés par les différents services météorologiques. Le téphigramme est utilisé en particulier en Grande-Bretagne et au Canada. On s'intéresse ici à l’émagramme ou plus précisément à l’émagramme oblique qui a été développé aux Etats-Unis ("Skew-T diagram").

- On utilisera ici les fonctionnalités du module Metpy : https://unidata.github.io/MetPy/latest/index.html.
- Pour la lecture de fichier texte on utilisera le module pandas : https://pandas.pydata.org/

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.gridspec as gridspec

import numpy as np
import pandas as pd

import metpy.calc as mpcalc
from metpy.plots import SkewT, Hodograph
from metpy.units import units
from metpy.plots import colortables

import warnings
warnings.filterwarnings("ignore")


# Préambule 1 : atmosphère standard

<div class="alert alert-warning">
<b>Coder la fonction de calcul de l'altitude à partir de la pression en atmosphère standard (OACI) : </b>
<br>
$$z = \frac{T_0}{\Gamma}[1-\frac{p}{p_0}^\frac{R\Gamma}{g}]$$
    
<br>

$$T_0=288 K$$
$$p_0=1013.25 hPa$$
$$g=9.8 m.s^{−2}$$
$$R_d=287 J.kg^{−1}.K^{−1}$$
$$\Gamma=0.0065 K.m^{−1}$$

</div>

In [ ]:
g = 9.8  # acceleration due to gravity
Rd = 287  # gas constant for dry air
gamma = 0.0065 # K/m
T0 = 288  # reference temperature in Kelvin
p0 = 1013.25  # reference surface pressure in hPa

def std_atm(pres):
    return

In [ ]:
pres=np.linspace(10,1000,100)
alt=std_atm(pres)
print('Pression (hPa)')
print(pres)
print('Altitudes OACI (m)')
print(alt)

fig = plt.figure(figsize=(6, 8))
plt.plot(pres, alt)
plt.grid()
plt.xlabel('Pression (hPa)')
plt.ylabel('Altitude (m)')
plt.title('Atmosphère standard')
plt.show()

fig.savefig('OACI.png')


# Préambule 2 : loi de Laplace et réduction de la pression au niveau de la mer

$$z_{2} - z_{1}= \frac{R_{d} \bar T_{v}}{g}ln\left(\frac{p_{1}}{p_{2}}\right)$$

<div class="alert alert-warning">
    
<b>Coder la fonction de calcul de la température virtuelle moyenne entre une station et le niveau de la mer connaissant l'altitude et la température virtuelle de la station :</b>

$$\bar T_{v} = \frac{1}{2}(T_{v}{mer}+T_{v}{station})= T_{v}{station} +\frac{\Gamma z}{2}$$

<br>
    
<b>Coder la fonction de réduction de la pression au niveau de la mer connaissant l'altitude et la pression de la station et la température virtuelle moyenne entre la station et le niveau de la mer :</b>

$$P_{mer} = P_{station} exp\left(\frac{g z}{R_d\bar T_{v}}\right)$$

<br>

$$g=9.8 m.s^{−2}$$
$$R_d=287 J.kg^{−1}.K^{−1}$$
$$\Gamma=0.0065 K.m^{−1}$$

</div>

In [ ]:
g = 9.8  # acceleration due to gravity
Rd = 287  # gas constant for dry air
gamma = 0.0065 # K/m

def Tvbar(Tv, z):
    return

def Pmer(z, p_stat, Tvbar):
    return

In [ ]:
z=390
p_stat=910
Tv=296

Tvbar=Tvbar(Tv,z)
Pmer=Pmer(z,p_stat,Tvbar)

print('Température virtuelle moyenne de la couche : ', Tvbar)
print('Pression réduite au niveau de la mer : ', Pmer)

# Ouverture du fichier du radiosondage

In [ ]:
col_names = ['pressure', 'temperature', 'dewpoint']
df = pd.read_fwf('sounding.txt',skiprows=4, usecols=[0, 1, 2], names=col_names)
df = df.dropna(subset=('temperature', 'dewpoint'), how='all').reset_index(drop=True)

p = df['pressure'].values * units.hPa
T = df['temperature'].values * units.degC
Td = df['dewpoint'].values * units.degC

print(df)

# Epaisseur du sondage

<div class="alert alert-warning">
<b>Coder la fonction de calcul de l'épaisseur du sondage : </b>
<br>
$$\Delta Z=Z_2 - Z_1 = -\frac{R_d}{g} \int_{p_1}^{p_2} T_v d\ln p$$
<br> On assimilera $T_v$ à $T$
<br> On utilisera la fonction d'intégration trapz de numpy : https://numpy.org/doc/stable/reference/generated/numpy.trapz.html

</div>

In [ ]:
def deltaz(T,p):
    return

In [ ]:
T2=np.array(T)+273.15
p2=np.array(p)
dz=deltaz(T2,p2)
p0=np.array(p[0]).astype(int)
p1=np.array(p[-1]).astype(int)
print('Epaisseur (en mètres) du sondage '+str(p0)+'-'+str(p1)+' hPa :', dz)

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.thickness_hydrostatic.html

In [ ]:
rh=mpcalc.relative_humidity_from_dewpoint(T, Td)
r=mpcalc.mixing_ratio_from_relative_humidity(p, T, rh)


thickness=mpcalc.thickness_hydrostatic(p, T, r)
print('Epaisseur du sondage '+str(p0)+'-'+str(p1)+' hPa :', thickness)

# Mesure de l'humidité
------------

Calculs effectués grâce aux fonctions de la bibliothèque calc de Metpy :
https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html

- Humidité relative (hygrométrie) : rapport de la pression partielle de la vapeur d'eau sur la pression de vapeur saturante à la même température : $$rh = \frac{e(T_d)}{e_s(T)}$$
- Humidité spécifique : mesure de l'humidité absolue. Rapport de la masse de vapeur d'eau sur la masse d'air humide. À ne pas confondre avec le rapport de mélange, qui est le rapport de la masse de vapeur d'eau sur la masse d'air sec : $$q  = \frac{0.622 e}{p-0.378 e}$$
- Rapport de mélange : mesure de l'humidité absolue. Rapport de la masse de vapeur d'eau sur la masse d'air sec : $$r = (rh)(r_s)$$ $$r = \frac{q}{1-q}$$
- Eau précipitable : intégrale verticale de l'humidité spécifique : $$-\frac{1}{\rho_l g} \int\limits_{p_\text{bottom}}^{p_\text{top}} q dp$$

In [ ]:
pw_calc=True

In [ ]:
rh=mpcalc.relative_humidity_from_dewpoint(T, Td)
print('Humidité relative (%)')
print(np.array(rh)*100)
print('*******************')

q=mpcalc.specific_humidity_from_dewpoint(p, Td)
print('Humidité specifique ((g/kg)')
print(np.array(q)*1000)
print('*******************')

#r=mpcalc.mixing_ratio_from_relative_humidity(rh, T, p) # old Metpy
r=mpcalc.mixing_ratio_from_relative_humidity(p, T, rh)
print("Rapport de mélange à partir de l'humidité relative (g/kg)")
print(np.array(r)*1000)
print('*******************')

r2=mpcalc.mixing_ratio_from_specific_humidity(q)
print("Rapport de mélange à partir de l'humidité spécifique (g/kg)")
print(np.array(r2)*1000)
print('*******************')

if pw_calc:
    pw=mpcalc.precipitable_water(p, Td)
    print('Eau précipitable (mm)')
    print(np.array(pw))
    print('*******************')

# Calculs de températures
------------

Calculs effectués grâce aux fonctions de la bibliothèque calc de Metpy :
https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html

- Température virtuelle : température qu'aurait une particule d'air sec qui possèderait la même masse volumique et la même pression qu'une particule d'air humide. L'air humide étant plus léger que l'air sec à la même pression et température, l'air humide a donc une température virtuelle plus élevée que l'air sec : $$T_v = T \frac{1 + 1.608\text{r}}{1 + \text{r}}$$
- Température du thermomètre mouillé : température d'une particule si on évapore l'eau liquide jusqu'à saturation à pression constante. La différence entre la température du thermomètre mouillé et la température et d'autant plus faible que l'hygrométrie de l'air est élevée : $$T_w$$
<img src="normand.gif" alt="drawing" width="200"/>
- Température potentielle : température qu'aurait la particule si on la déplace adiabatiquement au niveau de pression de référence 1000hPa. La température potentielle permet de comparer des particules d'air provenant d'altitudes différentes : $$\Theta = T (P_0 / P)^\kappa$$
- Température potentielle virtuelle : température potentielle qu'aurait de l'air sec qui possèderait la même masse volumique et la même pression que l'air humide (on l'utilise parfois pour remplacer la masse volumique dans les calculs de la flotabilité) : $$\Theta_v = \Theta \frac{\text{w} + \epsilon}{\epsilon\,(1 + \text{w})}$$
- Température potentielle équivalente : température d'une particule d'air à laquelle on aurait enlevé toute sa vapeur d'eau par un processus adiabatique et qu'on aurait ramené au niveau de pression de référence 1000hPa. La température potentielle équivalente permet de comparer des particules d'air ayant différents contenus en vapeur d'eau et provenant d'altitudes différentes : $$T_{L}=\frac{1}{\frac{1}{T_{D}-56}+\frac{ln(T_{K}/T_{D})}{800}}+56$$ $$\theta_{DL}=T_{K}\left(\frac{1000}{p-e}\right)^k
\left(\frac{T_{K}}{T_{L}}\right)^{.28r}$$ $$\theta_{E}=\theta_{DL}\exp\left[\left(\frac{3036.}{T_{L}}
-1.78\right)*r(1+.448r)\right]$$

In [ ]:
Tv=mpcalc.virtual_temperature(T, r)
print('Temperature virtuelle (°C)')
print(np.array(Tv)-273.15)
print('*******************')

Tw=mpcalc.wet_bulb_temperature(p, T, Td)
print('Temperature du thermomètre mouillé (°C)')
print(np.array(Tw))
print('*******************')

theta=mpcalc.potential_temperature(p, T)
print('Temperature potentielle (°C)')
print(np.array(theta)-273.15)
print('*******************')

thetav=mpcalc.virtual_potential_temperature(p, T, r)
print('Temperature potentielle virtuelle (°C)')
print(np.array(thetav)-273.15)
print('*******************')

thetae=mpcalc.equivalent_potential_temperature(p, T, Td)
print('Temperature potentielle équivalente (°C)')
print(np.array(thetae)-273.15)
print('*******************')

# Tracé du radiosondage sur l'émagramme
---------------------

Axe x : isothermes (angle de 45° avec les horizontales figurant des lignes isobares)
Axe y : isobares (échelle log)

1. Création d'un objet ``SkewT`` et spécification de l'angle entre isothermes et isobares
2. Tracé de la courbe d'état (P, T) en noir.
3. Tracé de la courbe des points de rosée (P, Td) : en orange
4. Tracé de la courbe de température du thermomètre mouillé (courbe bleue) (P, Tw) en bleu
5. Tracé de l'isotherme 0°C en cyan
6. Tracé des adiabatiques sèches (iso-θ) en vert
7. Tracé des adiabatiques humides (pseudo-adiabatiques saturées) en vert et des lignes d'égal rapport de mélange saturant en marron.

In [ ]:
fig = plt.figure(figsize=(9, 12))

skew = SkewT(fig, rotation=45)

skew.plot(p, T, 'black', linewidth=2)
skew.plot(p, Td, 'orange', linewidth=2)
skew.plot(p, Tw, 'blue', linewidth=2)

skew.ax.set_ylim(1020, 100)
skew.ax.set_xlim(-30, 40)

skew.ax.axvline(0, color='cyan', linestyle='--', linewidth=2)
skew.plot_dry_adiabats(colors='green', linestyle='-')
skew.plot_moist_adiabats(colors='green', linestyle='--')
skew.plot_mixing_lines(colors='brown', linestyle='--')

plt.show()

fig.savefig('sounding1.png')

# Diagnostics associés à la stabilité pour une particule issue de la base du sondage

Fonctionnalités de la bibliothèque calc de Metpy

- LCL = Lifting Condensation Level : niveau de condensation
- LFC = Level of Free Convection : niveau de convection libre
- LNB (EL) = Level of Neutral Buoyancy, Equilibrium Level : niveau de flottabilité neutre
- CAPE = Convective Available Potential Energy : énergie potentielle convective susceptible d’être transformée en énergie cinétique dans les mouvements ascendants
- CIN = Convection INhibition : énergie qu’il faut fournir à la particule pour qu’elle atteigne le niveau de convection libre

In [ ]:
# Calculate the parcel profile.
parcel_prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')

# Calculate the LCL
lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])
print('Point de condensation')
print(lcl_pressure, lcl_temperature)
print('*******************')

# Calculate the LFC
lfc_pt = mpcalc.lfc(p, T, Td, parcel_prof)
print('Niveau de convection libre')
print(lfc_pt)
print('*******************')

# Calculate the EL (LNB)
el_pressure, el_temperature = mpcalc.el(p, T, Td, parcel_prof)
print('Niveau de flottabilité neutre')
print(el_pressure, el_temperature)
print('*******************')

# Calculate CAPEs CINs
cape, cin = mpcalc.cape_cin(p, T, Td, parcel_prof)
print('CAPE')
print(cape)
print('*******************')
print('CIN')
print(cin)

# Analyse de la stabilité sur l'émagramme

- LCL : point noir.
- LFC : point rouge.
- LNB : point vert.
- CAPE : aire en rouge.
- CIN : aire en bleu.

In [ ]:
fig = plt.figure(figsize=(9, 12))

skew = SkewT(fig, rotation=45)

skew.plot(p, T, 'black')
skew.plot(p, Tw, 'blue')
skew.ax.set_ylim(1020, 100)
skew.ax.set_xlim(-30, 40)

skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
skew.plot(el_pressure, el_temperature, 'ko', markerfacecolor='green')
skew.plot(lfc_pt[0],lfc_pt[1], 'ko', markerfacecolor='brown')
skew.plot(p, parcel_prof, 'red', linewidth=2)
skew.shade_cin(p, T, parcel_prof)
skew.shade_cape(p, T, parcel_prof)
skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
skew.plot_dry_adiabats(colors='green', linestyle='-')
skew.plot_moist_adiabats(colors='green', linestyle='--')
skew.plot_mixing_lines(colors='brown', linestyle='--')

plt.show()

fig.savefig('sounding2.png')

# Diagnostics associés à la stabilité pour la particule la plus instable

Comme précédemment mais pour la particule la plus instable sur le profil.

In [ ]:
# Locate the most unstable parcel
mu_pressure, mu_temperature, mu_td, mu_index = mpcalc.most_unstable_parcel(p, T, Td)
print('Particule la plus instable')
print(mu_pressure, mu_temperature)
print('*******************')

# Calculate the most unstable parcel profile.
muparcel_prof = mpcalc.parcel_profile(p[mu_index:], T[mu_index], Td[mu_index]).to('degC')

# Calculate the LCL of most instable parcel
mulcl_pressure, mulcl_temperature = mpcalc.lcl(p[mu_index], T[mu_index], Td[mu_index])
print('Particule la plus instable : point de condensation')
print(mulcl_pressure, mulcl_temperature)
print('*******************')

# Calculate the LFC of MUP
mulfc_pt = mpcalc.lfc(p[mu_index:], T[mu_index:], Td[mu_index:], muparcel_prof)
print('Particule la plus instable : niveau de convection libre')
print(mulfc_pt)
print('*******************')

# Calculate the MUEL (LNB)
muel_pressure, muel_temperature = mpcalc.el(p[mu_index:], T[mu_index:], Td[mu_index:], muparcel_prof)
print('Particule la plus instable : niveau de flottabilité neutre')
print(muel_pressure, muel_temperature)
print('*******************')

# Calculate MUCAPE MUCIN
#mucape, mucin = mpcalc.most_unstable_cape_cin(p[mu_index:], T[mu_index:], Td[mu_index:])
mucape, mucin = mpcalc.most_unstable_cape_cin(p, T, Td)
print('CAPE de la particule la plus instable')
print(mucape)
print('*******************')
print('CIN de la particule la plus instable')
print(mucin)

# Calculate Mixed layer CAPE/CIN
mlcape, mlcin = mpcalc.mixed_layer_cape_cin(p, T, Td)
print('Mixed layer CAPE')
print(mlcape)
print('*******************')
print('Mixed layer CIN')
print(mlcin)

# Analyse de la stabilité sur l'émagramme

Comme précédemment mais pour la particule la plus instable sur le profil (repérée par un point jaune).

In [ ]:
fig = plt.figure(figsize=(9, 12))

skew = SkewT(fig, rotation=45)

skew.plot(p, T, 'black')
skew.plot(p, Tw, 'blue')
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-30, 40)

skew.plot(mu_pressure, mu_temperature, 'ko', markerfacecolor='yellow')
skew.plot(mulcl_pressure, mulcl_temperature, 'ko', markerfacecolor='black')
skew.plot(muel_pressure, muel_temperature, 'ko', markerfacecolor='green')
skew.plot(mulfc_pt[0],mulfc_pt[1], 'ko', markerfacecolor='brown')
skew.plot(p[mu_index:], muparcel_prof, 'red', linewidth=2)
skew.shade_cin(p[mu_index:], T[mu_index:], muparcel_prof)
skew.shade_cape(p[mu_index:], T[mu_index:], muparcel_prof)
skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

skew.plot_dry_adiabats(colors='green', linestyle='-')
skew.plot_moist_adiabats(colors='green', linestyle='--')
skew.plot_mixing_lines(colors='brown', linestyle='--')

plt.show()

fig.savefig('sounding3.png')

# Pour aller plus loin : exploitation du profil vertical de vent

In [ ]:
col_names = ['pressure', 'temperature', 'dewpoint', 'direction', 'speed']

df = pd.read_fwf('sounding.txt',
                 skiprows=4, usecols=[0, 1, 2, 3, 4], names=col_names)
df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)

p = df['pressure'].values * units.hPa
T = df['temperature'].values * units.degC
Td = df['dewpoint'].values * units.degC
Tw = mpcalc.wet_bulb_temperature(p, T, Td)
wind_speed = df['speed'].values * units.knots
wind_dir = df['direction'].values * units.degrees
u, v = mpcalc.wind_components(wind_speed, wind_dir)

print(df)
print("Vent zonal : ",u)
print("Vent méridien : ",v)

In [ ]:
# Calculate the parcel profile.
parcel_prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')
# Calculate the LCL
lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])
# Calculate the LFC
lfc_pt = mpcalc.lfc(p, T, Td, parcel_prof)
# Calculate the EL (LNB)
el_pressure, el_temperature = mpcalc.el(p, T, Td, parcel_prof)
# Calculate CAPEs CINs
cape, cin = mpcalc.cape_cin(p, T, Td, parcel_prof)

In [ ]:
fig = plt.figure(figsize=(9, 12))

skew = SkewT(fig, rotation=45)
skew.plot(p, T, 'black')
skew.plot(p, Tw, 'blue')
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1020, 100)
skew.ax.set_xlim(-30, 40)

skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
skew.plot(el_pressure, el_temperature, 'ko', markerfacecolor='green')
skew.plot(lfc_pt[0],lfc_pt[1], 'ko', markerfacecolor='brown')
skew.plot(p, parcel_prof, 'red', linewidth=2)
skew.shade_cin(p, T, parcel_prof)
skew.shade_cape(p, T, parcel_prof)
skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

skew.plot_dry_adiabats(colors='green', linestyle='-')
skew.plot_moist_adiabats(colors='green', linestyle='--')
skew.plot_mixing_lines(colors='brown', linestyle='--')

plt.show()

fig.savefig('sounding4.png')

L'**hodographe** est un outil très pratique qui permet d'étudier la variation du vent sur la verticale (cisaillement de vent), ingrédient très important dans la structuration de la convection profonde. L'hodograpne est tracé dans en géométrie polaire où les anneaux (lignes à rayon constant) représentent la vitesse du vent et l'angle indique la roses des vents (direction). On trace les vecteurs vent en commençant à partir du centre du domaine et on relie leurs sommets entre eux.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

h = Hodograph(ax, component_range=80.)
h.add_grid(increment=20)
h.wind_vectors(u, v)
norm, cmap = colortables.get_with_range('ir_rgbv', np.nanmin(wind_speed.m),
                                        np.nanmax(wind_speed.m))
h.plot_colormapped(u, v, wind_speed,cmap=cmap, norm=norm)
#h.plot(u, v)

plt.show()

fig.savefig('hodograph.png')

In [ ]:
fig = plt.figure(figsize=(12, 12))
gs = gridspec.GridSpec(3, 3)

skew = SkewT(fig, rotation=45, subplot=gs[:, :2])
skew.plot(p, T, 'black')
skew.plot(p, Tw, 'blue')
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1020, 100)
skew.ax.set_xlim(-30, 40)

skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
skew.plot(el_pressure, el_temperature, 'ko', markerfacecolor='green')
skew.plot(lfc_pt[0],lfc_pt[1], 'ko', markerfacecolor='brown')
skew.plot(p, parcel_prof, 'red', linewidth=2)
skew.shade_cin(p, T, parcel_prof)
skew.shade_cape(p, T, parcel_prof)
skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

skew.plot_dry_adiabats(colors='green', linestyle='-')
skew.plot_moist_adiabats(colors='green', linestyle='--')
skew.plot_mixing_lines(colors='brown', linestyle='--')

# Create a hodograph
ax = fig.add_subplot(gs[0, -1])
h = Hodograph(ax, component_range=80.)
h.add_grid(increment=20)
#h.plot_colormapped(u, v, wind_speed)  # Plot a line colored by wind speed
norm, cmap = colortables.get_with_range('ir_rgbv', np.nanmin(wind_speed.m),
                                        np.nanmax(wind_speed.m))

h.plot_colormapped(u, v, wind_speed,cmap=cmap, norm=norm)
#h.plot(u, v)

plt.show()

fig.savefig('sounding5.png')

# Radiosondages dans le monde

<div class="alert alert-warning">
<p><b>1) </b>Se rendre sur le site suivant : http://weather.uwyo.edu/upperair/sounding.html</p>
<p><b>2) </b>Télécharger un fichier de radiosondage</p>
<p><b>3) </b>Editer éventuellement le fichier pour qu'il puisse être lu correctement en Python</p>
<p><b>4) </b>Tracer l'émmagramme en s'inspirant des codes précédents.</p>
</div>